In [1]:
import cv2
import time
import pynvml
import pycuda.autoinit
from tqdm import tqdm
import pycuda.driver as cuda
import matplotlib.pyplot as plt
from trt_py.inferencer import TensorRTInfer
from trt_py.image_batcher import ImageBatcher

In [2]:
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

def print_gpu_stats():
    # Memory Info
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    gpu_used_mb = mem_info.used / 1024 ** 2
    temp = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU) 
    power = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000  # Convert from milliwatts to watts
    
    return gpu_used_mb, temp, power

# TensorRT Inference

In [3]:
def inference(model_path, image_path): 
    inference = TensorRTInfer(model_path)
    input_shape, input_dtype = inference.input_spec()

    batcher = ImageBatcher(
        input=image_path, 
        shape=input_shape, 
        dtype=input_dtype
    )

    total_data = 12475

    exec_time = 0
    gpu_usage = 0 
    temp = 0 
    power = 0

    all_data_time_start = time.time()
    for batch, images in tqdm(batcher.get_batch()):
        start_time = time.time()
        out = inference.infer(batch)
        end_time = time.time()
        exec_time += end_time - start_time

        gpu_usage_, temp_, power_ = print_gpu_stats()
        gpu_usage += gpu_usage_ 
        temp += temp_ 
        power += power_

    exec_time /= total_data 
    gpu_usage /= total_data 
    temp /= total_data 
    power /= total_data
    
    all_data_time_end = time.time()
    all_data_exec_time = all_data_time_end - all_data_time_start 

    result = { 
        "model" : model_path, 
        "total_execution_time" : all_data_exec_time, 
        "average_time_per_image" : exec_time, 
        "average gpu usage" : gpu_usage, 
        "average temperature" : temp, 
        "average power (watt)" : power
    }
    return result

In [4]:
image_path = "dataset/images"
model_path = "model/ghost_unet_1e-4--int8.engine" 

result = inference(model_path, image_path)

12475it [01:14, 167.54it/s]


In [6]:
result

{'model': 'model/ghost_unet_1e-4--int8.engine',
 'total_execution_time': 74.46552515029907,
 'average_time_per_image': 0.0029882674131221426,
 'average gpu usage': 258.52734375,
 'average temperature': 47.192625250501,
 'average power (watt)': 12.926220120240783}